In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import sys
import os
import pandas as pd
import time
import json

project_root = ".."
sys.path.append(project_root)

from src.support_etl import *


dap = pd.read_json(r'..\recetas_scrapper\dap.jsonl', lines=True)

with open("dap_leftoff.json", 'r', encoding='utf-8') as file:
    dap_leftoff = json.load(file)

print("Translating ingredients...")
for i in range(1+dap_leftoff["Position"],len(dap)):
    ingredients = dap["ingredientes"].loc[i]
    resultados = [
        " ".join(filter(None, [str(c) if c is not None else None, u, n]))
        for n, c, u in zip(ingredients['nombre'], ingredients['cantidad'], ingredients['unidad'])
    ]
    en_ingredients = [translate_es_en(e) for e in resultados]
    # print(en_ingredients)
    
    serving = dap["raciones"].loc[i]

    print("Getting nutrients...")

    nut_info, recipe_sum = get_nutrients(en_ingredients, int(serving))
    print(dap["url"][i])
    if isinstance(nut_info, int) and nut_info != 555:
        print("Error:", nut_info) 
        break # Detenemos el proceso si hay un error que no sea el de mal parseo de la receta
    elif isinstance(nut_info, int) and nut_info == 555:
        json.dump(dap_leftoff, open("dap_leftoff.json", "w")) # Guardar la posición actual
        continue # No guardamos la receta si no se pudo parsear
    # display(nut_info)
    # print(recipe_sum)
    
    dap_leftoff["Position"] = i
    

    filtered_nut_info = nut_info[nut_info["Weight (g)"] > 0].copy() # Filtrar ingredientes con peso > 0 para evitar subir ingredientes sin peso (macros nulos)
    columns_to_normalize = filtered_nut_info.columns[2:]  # Macros

    # Normalizar los registros para que sean cada 100 g
    
    filtered_nut_info[columns_to_normalize] = (
        filtered_nut_info[columns_to_normalize]
        .div(filtered_nut_info["Weight (g)"], axis=0)
        * 100
    )

    print("Connecting to database...")
    conn = connect_db()
    recipe_id = insert_recipe(conn, dap["titulo"].loc[i], translate_es_en(dap["titulo"].loc[i]), dap["titulo"].loc[i], dap["url"].loc[i],recipe_sum.get("weight"), recipe_sum, int(serving))
    for j in filtered_nut_info.index:
        ingredient_id = get_or_create_ingredient(conn, filtered_nut_info.loc[j].get("Ingredient").lower(), filtered_nut_info.loc[j], filtered_nut_info.loc[j].get("Ingredient").lower(), translate_en_es(filtered_nut_info.loc[j].get("Ingredient")).lower())
        insert_ingredient_recipe(conn, recipe_id, ingredient_id, float(filtered_nut_info.loc[j].get("Weight (g)")))
    json.dump(dap_leftoff, open("dap_leftoff.json", "w")) # Guardar la posición actual
    time.sleep(2)



print("End")

In [ ]:
def get_or_create_tag(conn, tagname):
    with conn.cursor() as cur:
    # Buscar el tag
        cur.execute("SELECT id FROM ingredients WHERE name = %s;", (tagname,))
        result = cur.fetchone()
    if result:
        # Si existe, devolver el tag
        return result[0]
    else:

In [32]:

import nltk

# nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

receta = "Lo primero es preparar la masa quebrada, para ello ponemos en un recipiente la harina, una cucharada de azúcar glas y el agua y luego, agregamos la mantequilla cortada en cubitos. Mezclamos bien hasta conseguir una masa homogénea. Al principio cuesta un poco porque la mantequilla estará dura, ayúdate de las manos para ir deshaciendo los cubos de mantequilla. Con esa masa forramos un molde de 22 cm y horneamos a 180ºC durante 10 - 15 minutos, es importante que pongas algo que pese sobre la masa, yo utilizo garbanzos pero puedes usar judías o unas bolillas de cerámica que venden para esto precisamente. Para hacer el relleno, muy sencillo, montamos las claras con una batidora de varillas, no hace falta que quede un merengue muy firme. Y luego le agregamos la harina de almendra y el azúcar. Mezclamos bien y ponemos sobre la masa quebrada previamente horneada. Volvemos a hornear durante 20-25 minutos a 180ºC. Veréis que toma un ligero tono tostado, eso es que está lista."
# receta = translate_es_en(receta)
pasos = sent_tokenize(receta) 

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\yanru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [33]:
for paso in pasos:
    print(paso)

Lo primero es preparar la masa quebrada, para ello ponemos en un recipiente la harina, una cucharada de azúcar glas y el agua y luego, agregamos la mantequilla cortada en cubitos.
Mezclamos bien hasta conseguir una masa homogénea.
Al principio cuesta un poco porque la mantequilla estará dura, ayúdate de las manos para ir deshaciendo los cubos de mantequilla.
Con esa masa forramos un molde de 22 cm y horneamos a 180ºC durante 10 - 15 minutos, es importante que pongas algo que pese sobre la masa, yo utilizo garbanzos pero puedes usar judías o unas bolillas de cerámica que venden para esto precisamente.
Para hacer el relleno, muy sencillo, montamos las claras con una batidora de varillas, no hace falta que quede un merengue muy firme.
Y luego le agregamos la harina de almendra y el azúcar.
Mezclamos bien y ponemos sobre la masa quebrada previamente horneada.
Volvemos a hornear durante 20-25 minutos a 180ºC.
Veréis que toma un ligero tono tostado, eso es que está lista.


In [34]:
print(pasos)

['Lo primero es preparar la masa quebrada, para ello ponemos en un recipiente la harina, una cucharada de azúcar glas y el agua y luego, agregamos la mantequilla cortada en cubitos.', 'Mezclamos bien hasta conseguir una masa homogénea.', 'Al principio cuesta un poco porque la mantequilla estará dura, ayúdate de las manos para ir deshaciendo los cubos de mantequilla.', 'Con esa masa forramos un molde de 22 cm y horneamos a 180ºC durante 10 - 15 minutos, es importante que pongas algo que pese sobre la masa, yo utilizo garbanzos pero puedes usar judías o unas bolillas de cerámica que venden para esto precisamente.', 'Para hacer el relleno, muy sencillo, montamos las claras con una batidora de varillas, no hace falta que quede un merengue muy firme.', 'Y luego le agregamos la harina de almendra y el azúcar.', 'Mezclamos bien y ponemos sobre la masa quebrada previamente horneada.', 'Volvemos a hornear durante 20-25 minutos a 180ºC.', 'Veréis que toma un ligero tono tostado, eso es que está 